In [79]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [80]:
listings_df = pd.read_csv('nyc_listings.csv')
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36923 entries, 0 to 36922
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            36923 non-null  int64  
 1   listing_url                                   36923 non-null  object 
 2   scrape_id                                     36923 non-null  float64
 3   last_scraped                                  36923 non-null  object 
 4   name                                          36910 non-null  object 
 5   description                                   35710 non-null  object 
 6   neighborhood_overview                         22510 non-null  object 
 7   picture_url                                   36923 non-null  object 
 8   host_id                                       36923 non-null  int64  
 9   host_url                                      36923 non-null 

In [81]:
# Removing dollar sign from price field
listings_df['price'] = listings_df['price'].str.replace("[$, ]", "").astype("float")

<ipython-input-81-81d63ba651f4>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  listings_df['price'] = listings_df['price'].str.replace("[$, ]", "").astype("float")


In [82]:
reviews_df = pd.read_csv('nyc_reviews.csv')

# Convert the date to datetime
reviews_df['date'] = pd.to_datetime(reviews_df['date'], format='%Y-%m-%d')
  
# Filter data between two dates
reviews_df = reviews_df.loc[(reviews_df['date'] >= '2019-01-01')
                     & (reviews_df['date'] < '2021-12-31')]
reviews_df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
42,2595,366477703,2019-01-02,165354250,Jacob,The apartment met expectations to how it was i...
43,2595,449815873,2019-05-07,168460710,Christopher,10 / 10 would stay again
44,2595,456489109,2019-05-21,119626931,Alex,Place was so cute and comfy! Host was great an...
45,2595,487972917,2019-07-14,60181725,Andrew,I'm glad I stayed here. I liked everything.<br...
46,2595,514642186,2019-08-21,33952280,Laura,"First of all I have to say, that Jennifer is n..."


In [83]:
# Rename id to listing_id for joining tables
listings_df.rename(columns={'id':'listing_id'}, inplace=True)

In [84]:
# Calculating revenue based on reviews
# Idea from Jingles Airbnb Analysis

booking_df = pd.merge(reviews_df, listings_df, on='listing_id')
booking_df['calculated_revenue'] = booking_df['price'] * booking_df['minimum_nights']

In [85]:
revenue_listing_df = booking_df[['listing_id','calculated_revenue']].groupby(['listing_id']).sum()

In [86]:
listings_df = pd.merge(listings_df, revenue_listing_df, on='listing_id', how='left')
listings_df.at[listings_df['calculated_revenue'].isnull(), 'calculated_revenue'] = 0

In [87]:
listings_df[['listing_id','number_of_reviews','minimum_nights','price','bedrooms','beds','calculated_revenue']].sort_values('calculated_revenue',ascending=False).head(5)


,listing_id,number_of_reviews,minimum_nights,price,bedrooms,beds,calculated_revenue
21157,33621795,49,30,9999.0,1.0,1.0,14698530.0
21160,33622378,48,30,9999.0,1.0,1.0,14398560.0
17425,26496505,224,31,1800.0,1.0,1.0,8928000.0
21159,33622308,22,30,9999.0,1.0,2.0,6599340.0
15876,22985168,26,72,10000.0,1.0,1.0,5040000.0


In [88]:
pd.set_option('display.max_rows', None)
listings_df[['listing_id','price']].groupby(['price']).count().sort_values('price', ascending=False)

,listing_id
price,
10000.0,7
9999.0,5
9990.0,1
9000.0,1
7314.0,1
7000.0,1
6500.0,2
6429.0,1
6143.0,1


In [89]:
high_price_df = listings_df[listings_df['price'] >= 7000]

high_price_df[['listing_id', 'listing_url', 'accommodates', 'number_of_reviews', 'price', 'calculated_revenue']]

,listing_id,listing_url,accommodates,number_of_reviews,price,calculated_revenue
3420,4308991,https://www.airbnb.com/rooms/4308991,2,9,9990.0,599400.0
3793,4737930,https://www.airbnb.com/rooms/4737930,4,1,9999.0,0.0
7234,9528920,https://www.airbnb.com/rooms/9528920,2,6,7314.0,0.0
10299,13925864,https://www.airbnb.com/rooms/13925864,4,28,10000.0,0.0
13983,20464496,https://www.airbnb.com/rooms/20464496,2,0,9000.0,0.0
15597,22436899,https://www.airbnb.com/rooms/22436899,4,0,10000.0,0.0
15876,22985168,https://www.airbnb.com/rooms/22985168,2,26,10000.0,5040000.0
18260,28135545,https://www.airbnb.com/rooms/28135545,1,0,7000.0,0.0
20089,31219800,https://www.airbnb.com/rooms/31219800,1,11,10000.0,4400000.0
21157,33621795,https://www.airbnb.com/rooms/33621795,2,49,9999.0,14698530.0


In [90]:
# Filter out price greater than 10000
listings_df = listings_df[listings_df['price'] < 7000]

In [91]:
listings_df[['listing_id','listing_url','number_of_reviews','minimum_nights','price','bedrooms','beds','calculated_revenue']].sort_values('calculated_revenue',ascending=False).head(5)


,listing_id,listing_url,number_of_reviews,minimum_nights,price,bedrooms,beds,calculated_revenue
17425,26496505,https://www.airbnb.com/rooms/26496505,224,31,1800.0,1.0,1.0,8928000.0
14648,21427176,https://www.airbnb.com/rooms/21427176,105,365,159.0,3.0,6.0,3656205.0
22075,35220744,https://www.airbnb.com/rooms/35220744,123,80,305.0,1.0,1.0,3001200.0
7591,9836940,https://www.airbnb.com/rooms/9836940,52,364,175.0,2.0,3.0,2611700.0
3766,4717296,https://www.airbnb.com/rooms/4717296,97,180,250.0,1.0,3.0,2475000.0


In [92]:
listings_df[['listing_id','minimum_nights']].groupby(['minimum_nights']).count().sort_values('minimum_nights', ascending=False)

,listing_id
minimum_nights,
1250,1
1124,1
1000,1
999,1
500,5
480,1
400,2
370,1
366,1


In [93]:
# Filter out min nights greater than 500
listings_df = listings_df[listings_df['minimum_nights'] <= 500]